In [1]:
from mothernet.prediction.mothernet_additive import MotherNetAdditiveClassifier
from mothernet.utils import get_mn_model
import numpy as np

device = "cpu"
model_str = "baam_H512_Dclass_average_e128_nsamples500_numfeatures20_padzerosFalse_03_14_2024_15_03_22_epoch_1520.cpkt"
model_path = get_mn_model(model_str)

additive = MotherNetAdditiveClassifier(path=model_path, device=device)

In [2]:
from sklearn.datasets import fetch_openml, load_iris
data = fetch_openml("prnn_crabs")
# data = load_iris()
# print(data)

In [3]:
from sklearn.model_selection import train_test_split
from interpret.glassbox import ExplainableBoostingClassifier

ebm_main = ExplainableBoostingClassifier(max_bins=64, interactions=0)
X_train, X_test, y_train, y_test = train_test_split(data.data.drop(columns="index"), data.target, stratify=data.target, train_size=0.5, random_state=13)

ebm_main.fit(X_train, y_train)

In [4]:
X_train_bool = X_train.copy()
X_train_bool['sex'] = (X_train['sex'] == "Female").astype(int)

X_test_bool = X_test.copy()
X_test_bool['sex'] = (X_test['sex'] == "Female").astype(int)

additive.fit(np.array(X_train_bool), y_train)

In [5]:
print(f"EBM: {ebm_main.score(X_test, y_test)}")
print(f"MotherNet: {additive.score(np.array(X_test_bool), y_test)}")

In [6]:
from interpret import show
ebm_global = ebm_main.explain_global()
show(ebm_global)

In [9]:
show(ag)